<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess-GoogleScholar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open Access - Google Scholar
La base de datos oaudea.json que fue creada con: [oaudea.ipynb](./oaudea.ipynb)

Está basada en la combinación de:

* Web of Science con Scopus y Scielo (Web of science) para la extración de la columna de DOIS e ISSNs de las revistas  de los artículos de la UdeA
* oadoi.org para la identificación de artículos Open Access
* DOAJ para obetener la información del APC basada en la identificación de los ISSNs de las revistas
* Búsqueda en google scholar (GS) de cada artículo basada en DOI y en título

In [0]:
import os
if os.getcwd()=='/content':
    !pip install openpyxl xlrd unidecode python-levenshtein wosplus > /dev/null

In [0]:
import pandas as pd
import wosplus as wp

In [3]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
oa_doi.json             = 1pLaEtyM-aOhjsB6twr05OhyJToIM7h7e
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd

Writing drive.cfg


In [0]:
oa=wp.wosplus('drive.cfg')

In [0]:
class names:
    def __init__(self,x):
        self.json=x+'.json'
        self.csv=x+'.csv'
        self.xlsx=x+'.xlsx'

In [0]:
oaudea=names('https://github.com/restrepo/medicion/raw/master/cienciometria/data/oaudea')

In [0]:
df=pd.read_json(oaudea.json)

In [8]:
df.shape

(7990, 39)

In [9]:
df.loc[[1]]

,APC,APC_USD,DOI,GS_Journal,GS_PDF,GS_Year,GS_abstract,GS_authors,GS_cites,GS_cites_link,...,journal_issns,journal_name,language,oa_locations,published_date,publisher,title,updated,year,z_authors
1,2950USD,2950,10.3389/fnhum.2017.00244,Frontiers in human …,https://www.frontiersin.org/articles/10.3389/f...,2017,… SHARE ON. Open Supplemental Data. Original R...,"SP Trujillo, S Valencia, N Trujillo…",4,/scholar?cites=10075282305931555032&as_sdt=200...,...,1662-5161,Frontiers in Human Neuroscience,en,"[{'host_type': 'publisher', 'pmh_id': None, 'u...",2017-05-23,Frontiers Media SA,Atypical Modulations of N170 Component during ...,2018-06-17T15:15:41.668136,2017,"[{'suffix': 'ST', 'family': 'Trujillo', 'given..."


Fix GS_cites

## Análisis

Número de artículos Open Access

In [10]:
dfoa=df[df.Open_Access!=''].reset_index(drop=True)
dfc=df[df.Open_Access==''].reset_index(drop=True) # close 
dfoa.shape[0],dfc.shape[0]

(3628, 4362)

### Gasto en OA

In [11]:
df.APC_USD.sum()

1001070

In [12]:
dfoa.APC_USD.sum()

1001070

### Citas OA

In [13]:
dfoa.GS_cites.sum()

81051

Falta bajar para no OA ?

In [14]:
dfc.GS_cites.sum()

91284

In [15]:
df.shape

(7990, 39)

### Years:

In [16]:
dfc[dfc.year>2002].year.value_counts()

2017    577
2016    449
2015    354
2014    346
2013    307
2012    261
2011    259
2010    250
2009    210
2008    180
2007    148
2006    112
2004     95
2005     88
2003     76
2018     39
Name: year, dtype: int64

In [17]:
dfoa[dfoa.year>2002].year.value_counts()

2017    574
2016    571
2015    503
2014    363
2013    240
2012    235
2011    202
2010    155
2009    141
2008     97
2007     78
2006     75
2005     50
2004     45
2018     38
2003     31
2099      1
Name: year, dtype: int64

## split bewteen GS good and bad

In [0]:
REBUILD=False
if REBUILD:
    datos=df.copy()
    dy=datos[datos.GS_quality_title==''].reset_index(drop=True)
    dn=datos[datos.GS_quality_title!=''].reset_index(drop=True)
    dy['GS_quality_title']=0
    datos=dy.append(dn).reset_index(drop=True)    
    datosmissing=datos[datos.GS_quality_title<0.9].reset_index(drop=True)
    datosgood=datos[datos.GS_quality_title>=0.9].reset_index(drop=True)

    dy=datosmissing[datosmissing.GS_quality_author==''].reset_index(drop=True)
    dn=datosmissing[datosmissing.GS_quality_author!=''].reset_index(drop=True)
    dy['GS_quality_author']=0
    datosmissing=dy.append(dn).reset_index(drop=True).reset_index(drop=True)

    datosgood=datosgood.append( 
        datosmissing[ ( (datosmissing.GS_quality_title>0.6 ) 
                       & (datosmissing.GS_quality_author>=0.9 ) )].reset_index(drop=True) ).reset_index(drop=True)

    datosmissing=datosmissing[ ~( (datosmissing.GS_quality_title>0.6 ) 
                       & (datosmissing.GS_quality_author>=0.9 ) )].reset_index(drop=True)

    datosgood.to_json('datosgood.json')

    datosmissing.to_json('datosmissing.json')
    df=datosgood.append(datosmissing).reset_index(drop=True)

In [0]:
REBUILD=False
if REBUILD:
    GSYES=datosgood
    GSNOT=datosmissing

In [0]:
if REBUILD:
    
    import pandas as pd

    noa=pd.read_json('gasnooa.json').reset_index(drop=True)


    GSNOT=GSNOT.drop(['GS_Journal', 'GS_Year', 'GS_abstract', 'GS_authors',
           'GS_cites', 'GS_cites_link', 'GS_profiles',  'GS_title',
           'GS_PDF','GS_old_title','GS_quality_author','GS_quality_title','GS_ref'],axis='columns')
    
    noags=wp.columns_add_prefix(noa,'GS').rename({'GS_DOI':'DOI'},axis='columns')

    GSNOTgs=GSNOT[GSNOT.DOI!=''].merge(noags[noags.DOI!=''],on='DOI',how='left').reset_index(drop=True)

    GSNOTgs=GSNOTgs.drop_duplicates('DOI').reset_index(drop=True)

    GSNOTgs['GS_cites']=GSNOTgs.GS_cites.fillna(0).astype(int)

    kk=GSYES.append( GSNOTgs,sort=True ).reset_index(drop=True)
    kk.fillna('').to_json('data/oaudea.json')

In [0]:
df.to_excel('oaudea.xlsx',index=False)